# <center> <span style="color:#f6f794"> 🧼 CLEANING DATA </span> </center>

_____

### <span style="color:#f6f794"> 📚 Importing libraries and setting displays </span>

In [1]:
import pandas as pd
import numpy as np
import requests
import gzip
from io import BytesIO
import os
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import time
import csv
import re


### <span style="color:#f6f794"> 📂 Opening files and create dataframes of them </span>

In [2]:
# Setting paths
current_dir = os.getcwd()                            # \sentiment-analyzer-inc\eda
print(current_dir)
parent_dir = os.path.dirname(current_dir)            # \sentiment-analyzer-inc
print(parent_dir)
data_dir = os.path.join(current_dir, "data")         # \sentiment-analyzer-inc\eda\data
print(data_dir)
cleaning_file_path = os.path.join(current_dir, "")   # \sentiment-analyzer-inc\eda\data\
print(cleaning_file_path)

c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\data
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\


In [3]:
GITHUB_CLEAN_URL = "https://raw.githubusercontent.com/Yael-Parra/sentiment-analyzer-inc/feature/data_youtube/etl/data/youtube_comments_-4GmbBoYQjE.csv.gz"

def load_airbnb_data_from_github(url):
    """
    Downloading and processing Airbnb data from GitHub repository.
    """
    print("⏯️ Downloading data from GitHub...")

    try:
        # Download the compressed CSV file from GitHub
        response = requests.get(url)
        response.raise_for_status()

        print("⏯️ bar_chart: Processing compressed file..")

        # Decompressing and reading the CSV file
        with gzip.open(BytesIO(response.content), 'rt', encoding='utf-8') as f:
            df = pd.read_csv(
                f,
                sep=',',                  # Specify the delimiter
                encoding='utf-8',         # Specify the encoding
                quoting=csv.QUOTE_MINIMAL # Specify the quoting behavior
                )

        print(f"✅ white_check_mark: Data downloaded succesfully!")
        
        return df

    except requests.exceptions.RequestException as req_err:
        print(f"⚠️ Network/Request Error while downloading data: {req_err}")
        return None
    except Exception as e:
        print(f"⚠️ General Error while processing data: {e}")
        return None

# Creating dataframe from GitHub URL
df_youtube_extract = load_airbnb_data_from_github(GITHUB_CLEAN_URL)

⏯️ Downloading data from GitHub...
⏯️ bar_chart: Processing compressed file..
✅ white_check_mark: Data downloaded succesfully!


___________
# <center> <span style="color:#f6f794">  🧹🧹🧹🧹🧹🧹🧹 START CLEANING 🧹🧹🧹🧹🧹🧹🧹 </center>
__________

In [4]:
df_youtube_extract.columns

Index(['threadId', 'commentId', 'videoId', 'author', 'authorChannelId',
       'isReply', 'parentCommentId', 'publishedAtComment', 'text',
       'likeCountComment', 'replyCount'],
      dtype='object')

In [5]:
df_youtube_extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   threadId            1000 non-null   object 
 1   commentId           1000 non-null   object 
 2   videoId             1000 non-null   object 
 3   author              1000 non-null   object 
 4   authorChannelId     1000 non-null   object 
 5   isReply             1000 non-null   bool   
 6   parentCommentId     27 non-null     object 
 7   publishedAtComment  1000 non-null   object 
 8   text                998 non-null    object 
 9   likeCountComment    1000 non-null   int64  
 10  replyCount          973 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 79.2+ KB


In [6]:
df_youtube_extract.sample(10)

,threadId,commentId,videoId,author,authorChannelId,isReply,parentCommentId,publishedAtComment,text,likeCountComment,replyCount
444,UgxXPSGZ9sovPy-bZg54AaABAg,UgxXPSGZ9sovPy-bZg54AaABAg,-4GmbBoYQjE,@Rajnita-b4w,UCbKZMmNM7XOWRuuesDo8h0Q,False,NaN,2025-06-24T12:53:17Z,Camping video so beautiful sir❤❤❤❤ love this c...,0,0.0
158,UgyHbBJigQ6GsyOgIOR4AaABAg,UgyHbBJigQ6GsyOgIOR4AaABAg,-4GmbBoYQjE,@JunniKassar,UCS2hyIDXrXa7s2biHuVjmCQ,False,NaN,2025-06-29T05:19:16Z,Hello,0,0.0
199,Ugyg2xsi2SjlRXcvd414AaABAg,Ugyg2xsi2SjlRXcvd414AaABAg,-4GmbBoYQjE,@Eltraviezobenja,UCMk4oW6c7rZPudnOuErTsaw,False,NaN,2025-06-28T13:26:38Z,Soy tu fan de perú🎉,0,0.0
909,UgyufhHlkBd0SjEvWC14AaABAg,UgyufhHlkBd0SjEvWC14AaABAg,-4GmbBoYQjE,@Hosna-d4e4g,UCJxmiO1RrWXIBeMeQvpFTbA,False,NaN,2025-06-18T04:47:18Z,আমি ওত সাবকাইব করেছি,0,0.0
224,UgyhzIV236QcyAwQQ5Z4AaABAg,UgyhzIV236QcyAwQQ5Z4AaABAg,-4GmbBoYQjE,@Kekob014,UCgHDCv1ED72WjyXyR3hQCyQ,False,NaN,2025-06-28T00:15:34Z,Bring back karl,0,0.0
492,UgyTNSdCkVKhg0_9dL14AaABAg,UgyTNSdCkVKhg0_9dL14AaABAg,-4GmbBoYQjE,@VictorMedeiros-y5s7k,UC0qyBGXMNWjb4vyKD8R5DpQ,False,NaN,2025-06-23T16:37:34Z,Oi eu sou muito seu fã 😊,0,0.0
781,UgxvLWMS2BQ9BFrzLwl4AaABAg,UgxvLWMS2BQ9BFrzLwl4AaABAg,-4GmbBoYQjE,@AlexlyGoated,UCi2JtFSMY_czMsMqMaP3Zjg,False,NaN,2025-06-19T23:22:07Z,I’m subscribed to all of your six channels,0,0.0
673,Ugxag3BzGLNrIoneCXp4AaABAg,Ugxag3BzGLNrIoneCXp4AaABAg,-4GmbBoYQjE,@Que-qk1uz,UCkU3qG37S33-pQ7y6EcusIg,False,NaN,2025-06-21T11:51:06Z,Thật tuyệt,0,0.0
844,UgzqjJ1woNp90WXp1vt4AaABAg,UgzqjJ1woNp90WXp1vt4AaABAg,-4GmbBoYQjE,@evonne0105,UCutuZPK9gk3DTL_ukdvBSSQ,False,NaN,2025-06-19T03:43:12Z,很假,0,0.0
388,UgxMIf0aIgjFIt7pbgp4AaABAg,UgxMIf0aIgjFIt7pbgp4AaABAg,-4GmbBoYQjE,@mpi7229,UCHnflxY4HQB_1KtwRdwB2vQ,False,NaN,2025-06-25T11:13:39Z,Aku bukan orang suku Maya,0,0.0


In [ ]:
df_youtube_extract.shape # (1000, 11)

(1000, 11)

#### <span style="color:#F3A006"> 1️⃣ Columns </span>

- Column's names format: lowercase with underscore as spaces

In [8]:
# Changin the names to lowercase and with a underscore to as spaces to make it easier to work with.
df_youtube_extract.columns = (
    df_youtube_extract.columns
    .str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True)  # Insert underscore before each uppercase (except start)
    .str.lower()                                       # Convert all to lowercase
    .str.strip()                                       # Remove leading and trailing spaces in case there were any
)

In [9]:
df_youtube_extract.columns

Index(['thread_id', 'comment_id', 'video_id', 'author', 'author_channel_id',
       'is_reply', 'parent_comment_id', 'published_at_comment', 'text',
       'like_count_comment', 'reply_count'],
      dtype='object')

#### <span style="color:#F3A006"> 2️⃣ Duplicates </span>

- There are no duplicate columns

In [ ]:
# Checking if there are duplicate columns
df_youtube_extract.columns.duplicated().any()        # np.False_
df_youtube_extract["comment_id"].duplicated().any()  # np.False_
df_youtube_extract["comment_id"].nunique()           # 7001
df_youtube_extract["video_id"].duplicated().any()    # np.True_
df_youtube_extract["video_id"].nunique()             # 1

#### <span style="color:#F3A006">  3️⃣  Nulls </span>

- Before imputing:
    - parent_comment_id:  qty of nulls:  973
    - text:               qty of nulls:    2
    - reply_count:        qty of nulls:   27

- After imputation:
    - parent_comment_id:
        - 
    - text:
        - 
    - reply_count: 
        - This value only makes sense for top-level comments, as replies do not receive further replies. Using -1 clearly distinguishes replies from root comments without mixing them with valid counts (like 0). It signals “not applicable” without being confused with an actual reply count.
     


In [ ]:
# Checking if there are nulls
df_youtube_extract.isnull().sum() # "parent_comment_id":  973     |  "text": 2      |    reply_count: 27
# Checking if there are empty rows
df_youtube_extract.isna().sum()
# Checkirng if there are empty strings
df_youtube_extract.eq('').sum()
# Checking if there are "nan" values
df_youtube_extract.eq('nan').sum()

["reply_count"]

In [ ]:
df_youtube_extract["reply_count"].nunique()  # 5
df_youtube_extract["reply_count"].unique()  # [638.,  nan,   0.,   1.,   2.,   7.]
df_youtube_extract["reply_count"].value_counts()

In [ ]:
df_youtube_extract["reply_count"] = df_youtube_extract["reply_count"].fillna(-1)

#### <span style="color:#F3A006"> 4️⃣ Changing the data types </span>

- DateTime: 
    - published_at

In [ ]:
df_youtube_extract.dtypes

In [ ]:
df_youtube_extract.head(2)

In [ ]:
# Changing data type to datetime for column  published_at_comment
df_youtube_extract["published_at_comment"] = pd.to_datetime(df_youtube_extract["published_at_comment"], errors='coerce')
df_youtube_extract["published_at_comment"].dtypes  # datetime64[ns, UTC]

#### <span style="color:#F3A006"> 5️⃣ Eliminating invisible row breakes </span>

In [ ]:
# Eliminating row breaks and extra spaces in the comments
df_youtube_extract["text"] = df_youtube_extract["text"].str.replace(r'\s+', ' ', regex=True).str.strip()

## <span style="color:#F3A006"> 🔟 Summary </span>

| Column Name          | Description                                                                                         |
|----------------------|-----------------------------------------------------------------------------------------------------|
| thread_id            | Unique identifier for the comment thread, grouping the original comment and its replies             |
| comment_id           | Unique identifier for each individual comment                                                       |
| video_id             | Unique identifier of the YouTube video where the comment was posted                                 |
| author               | Display name of the user who posted the comment                                                    |
| author_channel_id    | Unique identifier of the YouTube channel of the comment author                                      |
| is_reply             | Boolean indicating if the comment is a reply to another comment (True) or a top-level comment (False)|
| parent_comment_id    | Identifier of the parent comment if this comment is a reply; null or empty if top-level comment     |
| published_at_comment | Timestamp when the comment was originally published                                                 |
| text                 | The textual content of the comment                                                                 |
| like_count_comment   | Number of likes that the comment has received                                                      |
| reply_count          | Number of replies the comment has (only applicable for top-level comments)                          |


_______
# <center> <span style="color:#f6f794"> 💾💾💾💾💾 Saving in GitHub Compressed 💾💾💾💾💾</center>

In [ ]:
# # # Saving data in cleaning_file_path and in GitHub so it can be used in the next steps and the rest of the team can access it
# cleaning_file_path = os.path.join(data_dir, "youtube_dataset_cleaned.csv.gz")

# # Saving the cleaned DataFrame to a compressed CSV file
# df_raw_1.to_csv(
#     cleaning_file_path,
#     index=False,
#     compression='gzip',
#     sep=',',                  # Crucial: Specify delimiter even for compressed
#     encoding='utf-8',         # Crucial: Specify encoding
#     quoting=csv.QUOTE_MINIMAL # Crucial: Specify quoting behavior
# )
# print(f"✅ Data saved successfully at {cleaning_file_path}")

In [ ]:
# Saving the csv file in the data directory
cleaning_file_path = os.path.join(data_dir, "youtube_dataset_checking.csv")
# Saving the cleaned DataFrame to a CSV file
df_youtube_extract.to_csv(
    cleaning_file_path,
    index=False,
    sep=',',                  # Crucial: Specify delimiter
    encoding='utf-8',         # Crucial: Specify encoding
    quoting=csv.QUOTE_MINIMAL # Crucial: Specify quoting behavior
)
print(f"✅ Data saved successfully at {cleaning_file_path}")